In [1]:
import h5py
f = h5py.File('/home/aq/dataset/noisy_testset.hdf5', 'r')
list(f.keys())

['p232_001.wav',
 'p232_002.wav',
 'p232_003.wav',
 'p232_005.wav',
 'p232_006.wav',
 'p232_007.wav',
 'p232_009.wav',
 'p232_010.wav',
 'p232_011.wav',
 'p232_012.wav',
 'p232_013.wav',
 'p232_014.wav',
 'p232_015.wav',
 'p232_016.wav',
 'p232_017.wav',
 'p232_019.wav',
 'p232_020.wav',
 'p232_021.wav',
 'p232_022.wav',
 'p232_023.wav',
 'p232_024.wav',
 'p232_025.wav',
 'p232_027.wav',
 'p232_028.wav',
 'p232_029.wav',
 'p232_030.wav',
 'p232_031.wav',
 'p232_032.wav',
 'p232_033.wav',
 'p232_034.wav',
 'p232_035.wav',
 'p232_036.wav',
 'p232_037.wav',
 'p232_038.wav',
 'p232_039.wav',
 'p232_040.wav',
 'p232_041.wav',
 'p232_042.wav',
 'p232_043.wav',
 'p232_044.wav',
 'p232_045.wav',
 'p232_046.wav',
 'p232_047.wav',
 'p232_048.wav',
 'p232_049.wav',
 'p232_050.wav',
 'p232_051.wav',
 'p232_052.wav',
 'p232_053.wav',
 'p232_054.wav',
 'p232_055.wav',
 'p232_056.wav',
 'p232_057.wav',
 'p232_058.wav',
 'p232_059.wav',
 'p232_060.wav',
 'p232_061.wav',
 'p232_062.wav',
 'p232_063.wav

In [2]:
dset = f['p257_434.wav']

In [ ]:
import torch
import torch.nn as nn

# 第一种方法：Mask 类
class Mask(nn.Module):
    """Complex Ratio Mask"""
    def __init__(self, threshold=0.0001):
        super().__init__()
        self.threshold = threshold

    def forward(self, mask, spec):
        z = torch.complex(mask[:, 0], mask[:, 1])
        # z = torch.sigmoid(z)

        modulus = torch.abs(z)
        device = modulus.device 
        safe_modulus = torch.where(modulus == 0, torch.tensor(1e-10, dtype=modulus.dtype, device=device), modulus)
        mask_adjust = torch.where(modulus < self.threshold, self.threshold / safe_modulus, torch.ones_like(modulus))

        adjusted_z = z * mask_adjust
        mask_real = adjusted_z.real.unsqueeze(1)
        mask_imag = adjusted_z.imag.unsqueeze(1)
        mask = torch.cat((mask_real, mask_imag), dim=1)

        s_real = spec[:,0] * mask[:,0] - spec[:,1] * mask[:,1]
        s_imag = spec[:,1] * mask[:,0] + spec[:,0] * mask[:,1]
        s = torch.stack([s_real, s_imag], dim=1)
        return s

# 第二种方法：clamp_mask 函数
def clamp_mask(mask, threshold=0.0001):
    mask_complex = torch.complex(mask[:, 0], mask[:, 1])
    clamp_mag = torch.clamp(torch.abs(mask_complex), min=threshold)
    x = clamp_mag * torch.exp(1.0j * torch.angle(mask_complex))
    mask = torch.cat([x.real.unsqueeze(1), x.imag.unsqueeze(1)], dim=1)
    return mask

def apply_clamp_mask(mask, spec, threshold=0.0001):
    mask = clamp_mask(mask, threshold)
    s_real = spec[:, 0] * mask[:, 0] - spec[:, 1] * mask[:, 1]
    s_imag = spec[:, 1] * mask[:, 0] + spec[:, 0] * mask[:, 1]
    s = torch.stack([s_real, s_imag], dim=1)
    return s

# 生成测试数据
batch_size = 4
channels = 2
height = 10
width = 10

mask = torch.randn(batch_size, channels, height, width)
spec = torch.randn(batch_size, channels, height, width)

# 创建实例
mask_model = Mask()

# 应用第一种方法
output1 = mask_model(mask, spec)

# 应用第二种方法
output2 = apply_clamp_mask(mask, spec)

# 打印输出结果
print("Output of the first method (Mask class):")
print(output1)
print("\nOutput of the second method (clamp_mask function):")
print(output2)

# 比较输出结果
print("\nDifference between the two outputs:")
print(torch.abs(output1 - output2))


In [4]:
import torch
threshold = 5
mask = torch.randn(2,4)


In [5]:
threshold_tensor = threshold * torch.ones_like(mask)
neg_idx = torch.gt(mask, -threshold) & torch.le(mask, 0) # (-threshold, 0]
pos_idx = torch.gt(mask, 0) & torch.lt(mask, threshold) # (0, threshold)
mask = torch.where(neg_idx, -threshold_tensor, mask)
mask = torch.where(pos_idx, threshold_tensor, mask)